In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def roi(target):
    # define L and R ROI section.
    lpts = np.array([[280, 717],[570, 500],[620, 500],[380, 717]], np.int32)    
    rpts = np.array([[880, 717],[660, 500],[730, 500],[1020, 717]], np.int32)
    mask = np.zeros_like(target)
    cv2.fillConvexPoly(mask, lpts, (255,255,255))
    cv2.fillConvexPoly(mask, rpts, (255,255,255))
    masked = cv2.bitwise_and(target, mask)
    return masked

In [4]:
# video source input here.
video = './video_src/drive_trim.mp4'
# define window size
h, w = (640, 360)

cap = cv2.VideoCapture(video)

while True:

    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    blurred = cv2.GaussianBlur(gray, (5,5), 0)

    edges = cv2.Canny(blurred, 100,100) # canny edges
    roi_frame = roi(edges)  # roi edges
    view_frame = roi(blurred)   # blurred frame
    contours, hierachy = cv2.findContours(roi_frame, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(frame, contours, -1, (255,0,0), 0)

    # window panel ratio keeping.
    cv2.namedWindow("ROI_View", cv2.WINDOW_KEEPRATIO)   
    cv2.namedWindow("Canny", cv2.WINDOW_KEEPRATIO)
    cv2.namedWindow("Canny_Edges", cv2.WINDOW_KEEPRATIO)
    cv2.namedWindow("Frame", cv2.WINDOW_KEEPRATIO)

    cv2.imshow("ROI_View", view_frame)
    cv2.resizeWindow("ROI_View", h, w)
    cv2.imshow("Canny", edges)
    cv2.resizeWindow("Canny", h, w)
    cv2.imshow("Canny_Edges", roi_frame)
    cv2.resizeWindow("Canny_Edges", h, w)
    cv2.imshow('Frame', frame)
    cv2.resizeWindow("Frame", h, w)

    key = cv2.waitKey(13) 
    if key == 27:
        break   
    
cap.release()
cv2.destroyAllWindows()